In [1]:
import os 
import sys
import pandas as pd
import numpy as np
import confix
from sklearn import preprocessing
import time
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
#import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf


# Predictor 



In [2]:
# Combine all PDBs into a single dataframe
dfs = []
for filename in os.listdir(confix.PATH_FEATURES_RING):
    dfs.append(pd.read_csv(confix.PATH_FEATURES_RING + filename, sep='\t'))
df = pd.concat(dfs)


In [3]:
print(f'len of the data frame: {len(df)}')
df.head(10).T

len of the data frame: 735510


,0,1,2,3,4,5,6,7,8,9
pdb_id,2ia7,2ia7,2ia7,2ia7,2ia7,2ia7,2ia7,2ia7,2ia7,2ia7
s_ch,A,A,A,A,A,A,A,A,A,A
s_resi,43,100,101,94,70,33,100,95,108,72
s_ins,,,,,,,,,,
s_resn,T,E,A,V,T,I,E,T,I,G
s_ss8,-,T,G,E,H,H,T,E,E,H
s_rsa,0.07,0.34,0.396,0.07,0.099,0.006,0.34,0.43,0.0,0.214
s_up,18.0,13.0,5.0,16.0,12.0,18.0,13.0,10.0,21.0,3.0
s_down,10.0,5.0,13.0,14.0,10.0,11.0,5.0,22.0,14.0,18.0
s_phi,-1.96,-1.992,-1.028,-2.293,-1.011,-1.103,-1.992,-2.322,-1.776,-1.112


![DATA TABLE](data/images/table_data.png)

In [25]:
def drop_cols(df):
    df = df.dropna(inplace=True)

def Y_to_categorical(df):
    Y = df['Interaction'].astype('category')
    return Y

def transform_alphabetical_numerical(df):
    list = ['s_ch', 's_resn', 's_ss8', 's_ss3', 't_ch', 't_resn', 't_ss8', 't_ss3']
    label_encoder = preprocessing.LabelEncoder()
    for i in list:
        df[i] = label_encoder.fit_transform(df[i])

def create_X(df):
    df = df.drop(['Interaction', 's_ins', 't_ins', 'pdb_id'], axis=1)
    df = df.astype(float)
    X = df.rank(method='average', pct = True).round(1).astype('category') #pct = True
    return X

In [26]:
drop_cols(df)
Y = Y_to_categorical(df)
transform_alphabetical_numerical(df)
X = create_X(df)

In [27]:
X.head(10).T

,0,1,2,4,5,6,8,9,10,11
s_resi,60960.5,148470.5,149927.5,103582.5,44976.0,148470.5,160017.0,106639.5,56274.0,108174.0
s_resn,373766.0,84741.5,16697.5,373766.0,172071.5,84741.5,172071.5,135906.5,172071.5,228750.0
s_ss8,36419.0,442672.5,181340.5,296138.5,296138.5,442672.5,125776.5,296138.5,296138.5,296138.5
s_rsa,199228.5,347413.5,369748.0,222250.0,102710.5,347413.5,45540.5,289245.0,45540.5,352975.0
s_up,268635.0,163592.0,40764.5,145442.5,268635.0,163592.0,341384.5,21455.5,223585.5,95565.0
s_down,47212.0,5511.5,110725.5,47212.0,65361.5,5511.5,137129.5,253931.5,224092.5,110725.5
s_phi,89267.5,84962.5,383472.0,396705.0,302828.5,84962.5,113889.0,292491.5,363692.0,338671.5
s_psi,333438.0,277154.5,203781.0,164812.0,49737.5,277154.5,309090.5,184320.5,45278.0,75940.0
s_ss3,219852.0,219852.0,219852.0,219852.0,219852.0,219852.0,219852.0,219852.0,219852.0,219852.0
s_a1,241426.0,393375.5,171500.5,241426.0,49750.5,393375.5,49750.5,198106.5,49750.5,83967.0


In [28]:
Y.head(10).T

0     HBOND
1     HBOND
2     HBOND
4     HBOND
5       VDW
6       VDW
8     HBOND
9     HBOND
10    HBOND
11    HBOND
Name: Interaction, dtype: category
Categories (6, object): ['HBOND', 'IONIC', 'PICATION', 'PIPISTACK', 'SSBOND', 'VDW']

In [30]:
def forest_test(X, Y):
    X = X[0:100000]
    Y = Y[0:100000]
    X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, 
                                                        test_size = 0.30, 
                                                        random_state = 101)
    trainedforest = RandomForestClassifier(n_estimators=700).fit(X_Train,Y_Train)
    predictionforest = trainedforest.predict(X_Test)
    print(confusion_matrix(Y_Test,predictionforest))
    print(classification_report(Y_Test,predictionforest))

forest_test(X, Y)



[[16068   163     1   121     1  3282]
 [  333    88     0     0     0    79]
 [    7     0     0     0     0    24]
 [  102     0     0    73     0   438]
 [    1     0     0     0     3    57]
 [ 5235   100    18   392    38  3376]]
              precision    recall  f1-score   support

       HBOND       0.74      0.82      0.78     19636
       IONIC       0.25      0.18      0.21       500
    PICATION       0.00      0.00      0.00        31
   PIPISTACK       0.12      0.12      0.12       613
      SSBOND       0.07      0.05      0.06        61
         VDW       0.47      0.37      0.41      9159

    accuracy                           0.65     30000
   macro avg       0.28      0.26      0.26     30000
weighted avg       0.63      0.65      0.64     30000



In [43]:
def generate_one_hot(df, category):
    lables = []
    for i in range(0, len(df)):
        print(f'value {len(df["Interaction"][i])}')
        input()
        if df['Interaction'][i] == category:
            lables.append(1)
        else:
            lables.append(0)
    return lables

def generate_labels(df):
    category = df['Interaction'].astype('category')
    category = category.cat.categories.tolist()
    lables = {}
    for cat in category:
        lables[cat] = generate_one_hot(df, cat)
    return lables

labels = generate_labels(df)


value 1123


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().